In [7]:
from dotenv import load_dotenv
from openai import AsyncOpenAI

load_dotenv()
client = AsyncOpenAI()

In [17]:
# 6~8スピーチを1つのjsonに統合
import json
import re

output = {}

with open("../corpus/debate_scripts.json") as f:
    data = json.load(f)
    for round in data:
        title = re.sub(r'[\/:*?"<>|]', '', round["source"]["title"])  # Remove invalid filename characters
        
        input = []
        i = 0
        speech_num = len(round["speeches"])
        for speech_id, speech in enumerate(round["speeches"]):
            tmp_segment_list = []
            adus = speech["ADUs"]
            for adu_id, adu in enumerate(adus):
                tmp_segment_dict = {}
                tmp_segment_dict[i] = adu["transcript"]
                tmp_segment_list.append(tmp_segment_dict)
                i+=1
            input.append(tmp_segment_list)
        break #ここで最初の１試合だけ処理するようにしている
    counter_speech_pairs = []
    if len(input)==6:
        counter_speech_pairs.append(f"The 2nd proposition speech's argument units are {input[2]}. For its rebuttal, the speech targets the 1st opposition, whose argument units are {input[1]}.")
        # counter_speech_pairs.append(f"The 3nd proposition speech's argument units are {input[5]}. For its rebuttal, the speech targets the 1st, 2nd, and 3rd opposition, whose argument units are {input[1]}, {input[3]} and {input[4]}.")
        counter_speech_pairs.append(f"The 3nd proposition speech's argument units are {input[5]}. For its rebuttal, the speech targets the 1st opposition, whose argument units are {input[1]}, the 2nd opposition, whose argument units are {input[3]}, and the 3rd opposition, whose argument units are {input[4]}.")
        counter_speech_pairs.append(f"The 1st opposition speech's argument units are {input[1]}. For its rebuttal, the speech targets the 1st proposition, whose argument units are {input[0]}.")
        counter_speech_pairs.append(f"The 2nd opposition speech's argument units are {input[3]}. For its rebuttal, the speech targets the 1st proposition, whose argument units are {input[0]} and 2nd proposition, whose argument units are {input[2]}.")
        counter_speech_pairs.append(f"The 3rd opposition speech's argument units are {input[4]}. For its rebuttal, the speech targets the 1st proposition, whose argument units are {input[0]} and 2nd proposition, whose argument units are {input[2]}.")

[{0: "Thank you, Mr. Speaker and ladies and gentlemen in this house. Our side believes that we want to protect most moral people, every people, so every innocent people. And now another thing that's called the technology of hacking system or security camera is developing now and police can capture surprise information in the group of terrorists. So, thanks to those things, we can search those terrorists' move and act. "}, {1: "And the payment of ransom is money source of terrorist and criminal group. Huge rate of money source is ransom. Plus, criminal is very malicious. So, in many cases, even if victim's side pays huge money to the terrorist group and criminal groups, the victim will be killed by them. This is a very major case, but the family wants to protect their own family or own child, so they give money."}, {2: "Okay, let's move on to my argument. I have two arguments in this round. Firstly, I'd like to talk about long-term benefits, capturing those terrorists' source of money. 

In [5]:
for counter_speech_pair in counter_speech_pairs:
    print(counter_speech_pair)

The 2nd proposition speech's argument units are [{25: "Under the status quo, there are many incidents that are by terrorists. Many people are killed in airports or by Islamic State or ISIS. Many victims are existing and the victims' families suffer from the blacking of the family. They are killed by the terrorist, the service terrorists. So, we have to stop the situation. But after taking a bath, under the status quo, there are no solutions to stop. Uniquely, there is a high possibility to save the innocent citizens. So, we cannot stop the terrorists from killing people."}, {26: 'After taking this plan, we can minimize the situation because the almost payment of the terrorists is non-sum now. Many money is contributed to the terrorists by the citizens or government. Also, the individual issue. This is selfish and this makes the more serious situation around the world. So, we have to propose this proposal. Sorry.'}, {27: "Firstly, I will refute my previous speaker. The previous speaker 

In [16]:
print(len(input))

6


# テスト

In [8]:
results = []  # Create an empty list to store the results

for counter_speech_pair in counter_speech_pairs:
    user_prompt = f"Given the debate data: {counter_speech_pair}, please return a list of tuples representing the rebuttals. Each tuple should contain the id of the rebuttal source and the id of the rebuttal target. YOU MUST NOT RETURN ANYTHING OTHER THAN THE LIST OF TUPLES AND DO NOT INSERT A LINE BREAK."
    response = await client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a judge in the competitive debate and have to objectively analyze structures of arguments"},
            {"role": "user", "content": user_prompt}
        ]
    )
    results.append(response.choices[0].message.content)

In [15]:
import ast

rebuttals = []
for result in results:
    rebuttals+=(ast.literal_eval(result.replace("¥n",""))) #どうせadu_idとspeech_idの対応は別に求める必要があるので、ここではsrcのspeech_id情報は落とす

print(rebuttals)

[(27, 13), (27, 16), (32, 13), (63, 11), (64, 24), (65, 15), (66, 13), (67, 23), (11, 1), (13, 1), (14, 7), (15, 7), (20, 1), (21, 1), (23, 0), (24, 9), (34, 0), (34, 1), (35, 2), (36, 0), (37, 7), (38, 7), (39, 8), (46, 5), (47, 5), (51, 0), (52, 1), (52, 27), (53, 9), (54, 27), (54, 32), (55, 1), (55, 27), (55, 32), (56, 1), (57, 4), (58, 10), (59, 9), (60, 7)]


# 試行錯誤

In [33]:
user_prompt = f"Given the debate data: {counter_speech_pairs[2]}, please return a list of tuples representing the rebuttals. Each tuple should contain the id of the rebuttal source and the id of the rebuttal target. YOU MUST NOT RETURN ANYTHING OTHER THAN THE LIST OF TUPLES AND DO NOT INSERT A LINE BREAK."

response = await client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a judge in the competitive debate and have to objectively analyze structures of arguments"},
        {"role": "user", "content": user_prompt}
    ]
)

In [24]:
print(response.choices[0].message.content)
# 1,3,4と一緒くたにしているやつ
# Gov repです。人力では3つという判定だった！

Here is a list of tuples representing the rebuttals, where each tuple contains the id of the rebuttal source and the id of the rebuttal target:

1. (63, 11) - Addresses the point about citizens saving their families through ransom.
2. (64, 12) - Discusses the justification of criminalizing ransom payment in terms of criminal justice.
3. (65, 19) - Responds to the notion that negotiation should involve families rather than the police.
4. (66, 55) - Debunks the effectiveness of the opposition's model in preventing worst-case scenarios by explaining the government's superior plan.
5. (67, 23) - Argues the prioritization of protecting a larger number of people versus individual rights to self-defense.


In [32]:
print(response.choices[0].message.content)
# 1: 1st opposition, 3: 2nd opposition, 4: 3rd oppositionと区別して投げてるやつ

Based on the given information, here is a list of tuples representing the rebuttals in the debate:

1. (63, 11): The 3rd proposition rebuts the 1st opposition's claim about family saving and ransom negotiations.
2. (64, 12): The 3rd proposition addresses the justification and criminalization of ransom payments, partially rebutting the 1st opposition's undefined point about the science department.
3. (64, 13): The 3rd proposition refutes the 1st opposition's argument on the source of terrorist funding beyond ransom.
4. (65, 14): The 3rd proposition rebuts the 1st opposition's argument on the realism of police censoring citizens.
5. (66, 50): The 3rd proposition summarizes that their plan prevents worst-case accidents rebutting the general stance of the opposition.
6. (67, 19): The 3rd proposition prioritizes greater good protecting majority's right, rebutting 1st opposition's argument about government's duty of paying ransom in extreme cases like spies or soldiers.

This list includes t

In [34]:
print(response.choices[0].message.content)
# 1: 1st opposition, 3: 2nd opposition, 4: 3rd oppositionと区別して投げてるやつ、MUST NOT OTHER THAN入れてるver.

[(63, 34), (64, 34), (65, 34), (63, 11), (64, 11), (65, 11), (66, 52), (67, 53), (67, 12), (66, 51)]


In [38]:
for counter_speech_pair in counter_speech_pairs:
    user_prompt = f"Given the debate data: {counter_speech_pair}, please return a list of tuples representing the rebuttals. Each tuple should contain the id of the rebuttal source and the id of the rebuttal target. YOU MUST NOT RETURN ANYTHING OTHER THAN THE LIST OF TUPLES AND DO NOT INSERT A LINE BREAK."
    response = await client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a judge in the competitive debate and have to objectively analyze structures of arguments"},
            {"role": "user", "content": user_prompt}
        ]
    )
    print(response.choices[0].message.content)

[(27, 13), (28, 16), (31, 13)]
[(63, 13), (63, 16), (64, 12), (64, 27), (65, 19), (65, 20), (65, 28), (66, 11), (66, 12), (66, 13), (66, 14), (66, 18), (66, 19), (67, 23)]
[(11, 1), (13, 1), (14, 0), (14, 7), (14, 10), (16, 1), (17, 1), (18, 7), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 10)]
[(36, 0), (37, 2), (38, 2)]
[(52, 0), (55, 1), (56, 6), (59, 6), (60, 7)]


In [40]:
results = []  # Create an empty list to store the results

for counter_speech_pair in counter_speech_pairs:
    user_prompt = f"Given the debate data: {counter_speech_pair}, please return a list of tuples representing the rebuttals. Each tuple should contain the id of the rebuttal source and the id of the rebuttal target. YOU MUST NOT RETURN ANYTHING OTHER THAN THE LIST OF TUPLES AND DO NOT INSERT A LINE BREAK."
    response = await client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a judge in the competitive debate and have to objectively analyze structures of arguments"},
            {"role": "user", "content": user_prompt}
        ]
    )
    results.append(response.choices[0].message.content)  # Append the result to the list

In [41]:
# Print all results after the loop
for result in results:
    print(result)
# AND DO NOT INSERT A LINE BREAK.を入れてないやつ

[(27, 13), (27, 16), (32, 20)]
[
    (63, 13),
    (64, 24),
    (65, 18),
    (66, 13),
    (66, 16),
    (66, 38),
    (67, 23),
    (67, 49)
]
[(11, 1), (13, 1), (14, 3), (16, 1), (17, 3), (18, 3), (19, 3), (20, 9), (21, 1), (23, 1), (24, 3)]
[(34, 1), (36, 0), (37, 6), (38, 6), (39, 8), (40, 6), (41, 1), (41, 9), (42, 7), (43, 1), (45, 3), (47, 1), (47, 9), (48, 8), (49, 9)]
[(52, 0), (52, 1), (55, 8), (55, 9), (56, 6), (56, 7), (59, 7), (60, 7)]


## tempretureの検証

In [52]:
user_prompt = f"Given the debate data: {counter_speech_pairs[2]}, please return a list of tuples representing the rebuttals. Each tuple should contain the id of the rebuttal source and the id of the rebuttal target. YOU MUST NOT RETURN ANYTHING OTHER THAN THE LIST OF TUPLES AND DO NOT INSERT A LINE BREAK."

response = await client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a judge in the competitive debate and have to objectively analyze structures of arguments"},
        {"role": "user", "content": user_prompt}
    ],
    temperature=1.0,
    n=5
)

In [46]:
print("temperature=0.0")
for i in range(5):
    print(response.choices[i].message.content)

temperature=0.0
[(11, 1), (13, 1), (13, 6), (14, 7), (15, 7), (16, 1), (17, 7), (18, 7), (19, 7), (20, 9), (21, 9), (22, 9), (23, 9), (24, 9)]
[(11, 1), (13, 1), (13, 6), (14, 7), (15, 7), (16, 1), (17, 7), (18, 7), (19, 7), (20, 9), (21, 9), (22, 9), (23, 9), (24, 9)]
[(11, 1), (13, 1), (13, 6), (14, 7), (15, 7), (16, 1), (17, 7), (18, 7), (19, 7), (20, 9), (21, 9), (22, 9), (23, 9), (24, 9)]
[(11, 1), (13, 1), (13, 6), (14, 7), (15, 7), (16, 1), (17, 7), (18, 7), (19, 7), (20, 9), (21, 9), (22, 9), (23, 9), (24, 9)]
[(11, 1), (13, 1), (13, 6), (14, 7), (15, 7), (16, 1), (17, 7), (18, 7), (19, 7), (20, 9), (21, 9), (22, 9), (23, 9), (24, 9)]


In [48]:
print("temperature=1.0")
for i in range(5):
    print(response.choices[i].message.content)

temperature=1.0
[(11, 0), (13, 1), (13, 6), (15, 2), (15, 9), (17, 4), (20, 6), (21, 7), (21, 10), (23, 3), (23, 8), (24, 6), (24, 10)]
[(11, 1), (13, 1), (14, 0), (19, 9), (23, 9), (24, 3), (21, 9)]
[(11, 1), (13, 1), (14, 7), (15, 4), (16, 1), (17, 4), (18, 7), (19, 3), (20, 9), (21, 7), (22, 9), (23, 7), (24, 10)]
[(11, 1), (13, 1), (16, 1), (18, 7), (20, 9), (21, 9)]
[(11, 1), (11, 6), (11, 10), (11, 9), (13, 1), (13, 5), (14, 1), (14, 9), (14, 6), (15, 7), (16, 6), (18, 7), (19, 3), (20, 9), (20, 10), (21, 1), (21, 4)]


In [53]:
print("temperature=1.9")
for i in range(1):
    print(response.choices[i].message.content)

temperature=2.0
[(11,1), (11,9), (18,2), (12,2),(23,2), (23,6)]


Temperature Consistancy across responses

In [56]:
user_prompt = f"Given the debate data: {counter_speech_pairs[2]}, please return a list of tuples representing the rebuttals. Each tuple should contain the id of the rebuttal source and the id of the rebuttal target. YOU MUST NOT RETURN ANYTHING OTHER THAN THE LIST OF TUPLES AND DO NOT INSERT A LINE BREAK."

response = await client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a judge in the competitive debate and have to objectively analyze structures of arguments"},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.0
)

user_prompt = f"Given the debate data: {counter_speech_pairs[2]}, please return a list of tuples representing the rebuttals. Each tuple should contain the id of the rebuttal source and the id of the rebuttal target. Plus, explain the reason for each rebuttal identification."

response2 = await client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a judge in the competitive debate and have to objectively analyze structures of arguments"},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.0
)

print(response.choices[0].message.content)
print(response2.choices[0].message.content)

[(11, 1), (13, 1), (14, 7), (15, 7), (16, 1), (17, 7), (18, 7), (19, 7), (20, 9), (21, 9), (22, 9), (23, 9), (24, 9)]
Based on the provided argument units from the 1st opposition speech and the 1st proposition speech, here is a list of tuples representing the rebuttals, along with explanations for each identification:

1. **Rebuttal Source: 11, Rebuttal Target: 1**
   - **Reason:** The 1st opposition argues that prosecuting terrorists because they killed citizens is counterproductive if the fundamental purpose is against terrorism. This directly counters the 1st proposition's claim that ransom payments are a major source of income for terrorists and that paying ransom often results in the victim being killed.

2. **Rebuttal Source: 13, Rebuttal Target: 1**
   - **Reason:** The 1st opposition challenges the idea that ransom is a significant source of income for terrorists by providing examples like ISIS, whose income comes from other sources. This directly rebuts the 1st proposition's a

In [57]:
user_prompt = f"Given the debate data: {counter_speech_pairs[2]}, please return a list of tuples representing the rebuttals. Each tuple should contain the id of the rebuttal source and the id of the rebuttal target. YOU MUST NOT RETURN ANYTHING OTHER THAN THE LIST OF TUPLES AND DO NOT INSERT A LINE BREAK."

response = await client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a judge in the competitive debate and have to objectively analyze structures of arguments"},
        {"role": "user", "content": user_prompt}
    ],
    top_p=0.0
)

user_prompt = f"Given the debate data: {counter_speech_pairs[2]}, please return a list of tuples representing the rebuttals. Each tuple should contain the id of the rebuttal source and the id of the rebuttal target. Plus, explain the reason for each rebuttal identification."

response2 = await client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a judge in the competitive debate and have to objectively analyze structures of arguments"},
        {"role": "user", "content": user_prompt}
    ],
    top_p=0.0
)

print(response.choices[0].message.content)
print(response2.choices[0].message.content)

[(11, 1), (13, 1), (13, 6), (14, 7), (15, 7), (16, 1), (17, 7), (18, 7), (19, 7), (20, 9), (21, 9), (22, 9), (23, 9), (24, 9)]
Based on the provided data, here is a list of tuples representing the rebuttals from the 1st opposition speech to the 1st proposition speech, along with explanations for each identification:

1. **(11, 1)**: The 1st opposition argues that prosecuting terrorists is counterproductive if the fundamental purpose is against terrorism. This directly counters the 1st proposition's claim that ransom payments are a major source of funding for terrorists and that paying ransom is harmful because it funds further terrorism.

2. **(13, 1)**: The 1st opposition challenges the 1st proposition's assertion that ransom is a significant source of terrorist funding by providing examples like ISIS, whose income comes from other sources. This directly rebuts the 1st proposition's argument about the financial impact of ransom payments.

3. **(14, 7)**: The 1st opposition critiques t